In [1]:
from cellpose.gui import gui
from pipeline.test import run_gui

/opt/conda/envs/cp_dock/lib/python3.8/site-packages/pyqtgraph/colors/palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.12.1 detected.
  from ..Qt import QtGui


2024-02-17 19:07:41,263 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2024-02-17 19:07:41,265 [INFO] 
cellpose version: 	2.2.3 
platform:       	linux 
python version: 	3.8.18 
torch version:  	1.11.0


: 

In [5]:
run_gui()

2024-02-17 19:05:14,779 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2024-02-17 19:05:14,784 [INFO] 
cellpose version: 	2.2.3 
platform:       	linux 
python version: 	3.8.18 
torch version:  	1.11.0


: 

In [2]:
from qtpy.QtCore import Qt
from qtpy.QtGui import QFont
from qtpy.QtWidgets import QApplication, QLabel
from pyqt_vertical_tab_widget.verticalTabWidget import VerticalTabWidget
import sys

In [ ]:
from cellpose.io import logger_setup
global logger
logger, log_file = logger_setup()
# Always start by initializing Qt (only once per application)
warnings.filterwarnings("ignore")
app = QApplication(sys.argv)
icon_path = pathlib.Path.home().joinpath('.cellpose', 'logo.png')
guip_path = pathlib.Path.home().joinpath('.cellpose', 'cellpose_gui.png')
style_path = pathlib.Path.home().joinpath('.cellpose', 'style_choice.npy')
if not icon_path.is_file():
    cp_dir = pathlib.Path.home().joinpath('.cellpose')
    cp_dir.mkdir(exist_ok=True)
    print('downloading logo')
    download_url_to_file('https://www.cellpose.org/static/images/cellpose_transparent.png', icon_path, progress=True)
if not guip_path.is_file():
    print('downloading help window image')
    download_url_to_file('https://www.cellpose.org/static/images/cellpose_gui.png', guip_path, progress=True)
if not style_path.is_file():
    print('downloading style classifier')
    download_url_to_file('https://www.cellpose.org/static/models/style_choice.npy', style_path, progress=True)
icon_path = str(icon_path.resolve())
app_icon = QtGui.QIcon()
app_icon.addFile(icon_path, QtCore.QSize(16, 16))
app_icon.addFile(icon_path, QtCore.QSize(24, 24))
app_icon.addFile(icon_path, QtCore.QSize(32, 32))
app_icon.addFile(icon_path, QtCore.QSize(48, 48))
app_icon.addFile(icon_path, QtCore.QSize(64, 64))
app_icon.addFile(icon_path, QtCore.QSize(256, 256))
app.setWindowIcon(app_icon)

# models.download_model_weights() # does not exist
MainW(image=None)
ret = app.exec_()
sys.exit(ret)

In [3]:
from pipeline.gui import gui

/opt/conda/envs/cp_dock/lib/python3.8/site-packages/pyqtgraph/colors/palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.12.1 detected.
  from ..Qt import QtGui


In [4]:
app = QApplication(sys.argv)

myWindow = VerticalTabWidget()


tab1 = gui.MainW()

myWindow.addTab(tab1, 'Tab 1')


tab2 = QLabel()

tab2.setAlignment(Qt.AlignCenter)

tab2.setText(f'Widget 2')

tab2.setFont(QFont('Arial', 30, QFont.Bold))

myWindow.addTab(tab2, 'Tab 2')

myWindow.show()

sys.exit(app.exec_())

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'
QLayout: Attempting to add QLayout "" to MainW "", which already has a layout
Empty filename passed to function
Empty filename passed to function
ERROR: could not read file, 'NoneType' object has no attribute 'ndim'


GUI_INFO: loading image: 


AttributeError: 'NoneType' object has no attribute 'ndim'

SystemExit: 0

/opt/conda/envs/cp_dock/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
